In [2]:
!pip install -q transformers
!pip install -q pyTelegramBotAPI
#!pip install -q torch torchvision


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install -q tensorflow


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install -q Wikipedia


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import transformers
import telebot
import transformers
import torch # pytorch will be needed for BioBert
import tensorflow
import wikipedia 
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:


model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
bot_token = '6036588459:AAG65SB0NYDElctvhphsXnsYy0MpPs4l9yw'

bot = telebot.TeleBot(bot_token, parse_mode=None) # You can set parse_mode by default. HTML or MARKDOWN

In [7]:
nlp( 'What are the symptoms of COVID-19 ?','“Symptoms of COVID-19 are variable, but often include fever, cough, fatigue, breathing difficulties, and loss of smell and taste. Symptoms may begin one to fourteen days after exposure to the virus. At least a third of people who are infected do not develop noticeable symptoms.[9] Of those people who develop noticeable symptoms enough to be classed as patients, most (81%) develop mild to moderate symptoms (up to mild pneumonia), while 14% develop severe symptoms (dyspnea, hypoxia, or more than 50% lung involvement on imaging), and 5% suffer critical symptoms (respiratory failure, shock, or multiorgan dysfunction).[10] Older people are more likely to have severe symptoms. Some people continue to experience a range of effects—known as long COVID—for months after recovery, and damage to organs has been observed.[11] Multi-year studies are underway to further investigate the long-term effects of the disease.”')

[2023-03-02 04:42:42.918 pytorch-1-13-cpu-py39-ml-t3-medium-9140905751f3e451a2295c86a7c3:259 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None


{'score': 0.7451003789901733,
 'start': 54,
 'end': 128,
 'answer': 'fever, cough, fatigue, breathing difficulties, and loss of smell and taste'}

### Simple Logic: 
> This handler answers the question by doing the following:
1. Ask the user about a topic, search the Wiki for the most relavant information, and uses the .summary method to get the plain text summary of the page

2. Then, it asks for a specific question form the user, and passes the summary as context, the question as question to the "roberta-base-squad2" model from hugging face, and gets the answers


> wikipedia API documents: https://wikipedia.readthedocs.io/en/latest/code.html#module-wikipedia
> Squad2 RoBerta: https://huggingface.co/deepset/roberta-base-squad2

In [32]:

bot = telebot.TeleBot(bot_token, parse_mode=None) # You can set parse_mode by default. HTML or MARKDOWN

talk = 0


# this is the QA handler
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
	bot.reply_to(message, "Howdy, how are you doing?")
@bot.message_handler(func=lambda message: True)
def Question_Answering(message):
    
    global talk
    global su

    if talk == 0:
      bot.reply_to(message, 'tell me a subject you wanted to talk about')
      talk = 1
      return
    
    if talk == 1:
      results = wikipedia.search(message.text)
      summary = wikipedia.summary(results[0])
      su = summary
      bot.reply_to(message, summary)#'ask me a question about this topic')
      talk = 2
      return
    
    if talk == 2:
      a = nlp(context=su, question=message.text)
      bot.reply_to(message, list(a.values())[3])
      talk = 0
      return

In [ ]:
bot.infinity_polling()

2023-03-02 05:13:59,639 (__init__.py:960 MainThread) ERROR - TeleBot: "Infinity polling exception: Page id "2020 winter olympics" does not match any pages. Try another id!"
2023-03-02 05:13:59,641 (__init__.py:962 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/telebot/__init__.py", line 955, in infinity_polling
    self.polling(non_stop=True, timeout=timeout, long_polling_timeout=long_polling_timeout,
  File "/opt/conda/lib/python3.9/site-packages/telebot/__init__.py", line 1043, in polling
    self.__threaded_polling(non_stop=non_stop, interval=interval, timeout=timeout, long_polling_timeout=long_polling_timeout,
  File "/opt/conda/lib/python3.9/site-packages/telebot/__init__.py", line 1118, in __threaded_polling
    raise e
  File "/opt/conda/lib/python3.9/site-packages/telebot/__init__.py", line 1074, in __threaded_polling
    self.worker_pool.raise_exceptions()
  File "/opt/conda/lib/python3.9/sit